In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK：AutoML训练视频分类模型用于批量预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_video_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_video_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/sdk_automl_video_classification_batch.ipynb" target='_blank'>            
    <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 
在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用 Vertex AI SDK 创建视频分类模型，并使用 Vertex AI [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) 模型进行批量预测。

了解更多关于[视频数据分类](https://cloud.google.com/vertex-ai/docs/training-overview#classification_for_videos)的信息。

### 目标

在这个教程中，您将学习如何从Python脚本创建一个AutoML视频分类模型，然后使用Vertex AI SDK进行批量预测。
另外，您也可以使用`gcloud`命令行工具或通过Google Cloud控制台在线创建和部署模型。

本教程使用以下Google Cloud ML服务和资源:

- Vertex AI训练
- Vertex AI模型
- Vertex AI批量预测

执行的步骤包括:

- 创建一个Vertex数据集资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和使用在线预测之间有一个关键区别:

* 预测服务: 对整个实例集（即一个或多个数据项）进行按需预测，并实时返回结果。

* 批量预测服务: 对整个实例集进行排队（批量）预测，并在准备就绪时将结果存储在Cloud Storage存储桶中。

### 数据集

本教程使用的数据集是麻省理工学院[人体运动数据集](http://cbcl.mit.edu/publications/ps/Kuehne_etal_iccv11.pdf)中的高尔夫挥杆识别部分。本教程中使用的数据集版本存储在一个公共云存储桶中。训练模型可以预测高尔夫挥杆开始的帧数。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装最新版本的Vertex AI和Cloud Storage SDK for Python。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage

### 仅限协作：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

设置你的项目ID

**如果你不知道你的项目ID**，请尝试以下步骤：
- 运行 `gcloud config list`。
- 运行 `gcloud projects list`。
- 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI 工作台**
* 无需操作，因为您已经通过身份验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

协作，取消注释并运行。

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 请查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上授予您的服务账户云存储权限。

### 创建一个云存储桶

创建一个存储桶来存储中间产品，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [13]:
from google.cloud import aiplatform

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## 教程

现在您已经准备好开始创建您自己的AutoML视频分类模型。

#### 云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的MIT人体运动数据集的版本，使用一个CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中行数来计算示例的数量（`wc -l`），然后查看前几行。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

### 创建数据集

接下来，使用`VideoDataset`类的`create`方法创建数据集资源，该方法接受以下参数：

- `display_name`：数据集资源的人类可读名称。
- `gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name="MIT Human Motion",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.classification,
)

print(dataset.resource_name)

### 创建和运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练流水线，和2）运行该流水线。

#### 创建训练流水线

使用`AutoMLVideoTrainingJob`类创建一个AutoML训练流水线，其中包含以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：模型训练的任务类型。
  - `classification`：视频分类模型。
  - `object_tracking`：视频目标追踪模型。
  - `action_recognition`：视频动作识别模型。

In [ ]:
job = aiplatform.AutoMLVideoTrainingJob(
    display_name="hmdb",
    prediction_type="classification",
)

print(job)

#### 运行训练管道

接下来，您可以通过调用`run`方法来运行作业，以启动训练作业，并指定以下参数：

- `dataset`：用于训练模型的数据集资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（保留数据）的数据集百分比。

完成`run`方法后将返回训练模型资源。

训练管道的执行可能需要超过3个小时。

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name="hmdb",
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

## 检查模型评估分数
在您的模型训练完成后，您可以查看其评估分数。

您可以使用 Vertex AI 模型资源的 `get_model_evaluation` 方法来检查模型的评估结果。

与 Vertex AI 数据集一样，您可以使用部署模型时创建的模型变量的引用，或者可以通过使用模型的显示名称从项目中所有模型的列表中筛选。

In [ ]:
# Get model resource ID using the display_name
models = aiplatform.Model.list(filter="display_name=hmdb")

if len(models) != 0:

    # Get the model object
    model_rsc_name = models[0].resource_name
    print("Model resource name:", model_rsc_name)
    model = aiplatform.Model(model_rsc_name)

    # Print the model evaluation
    model_eval = model.get_model_evaluation()
    print(model_eval.to_dict())

发送一个批量预测请求

向部署的模型发送一个批量预测。

获取测试项目

现在对您的 Vertex AI 模型进行批量预测。使用数据集中的任意示例作为测试项目。
不必担心这些示例很可能已在训练模型中使用，因为本教程仅关于如何进行批量预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2

if len(test_items[0]) == 5:
    _, test_item_1, test_label_1, _, _ = str(test_items[0]).split(",")
    _, test_item_2, test_label_2, _, _ = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1, _, _ = str(test_items[0]).split(",")
    test_item_2, test_label_2, _, _ = str(test_items[1]).split(",")


print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 创建批处理输入文件

现在创建一个批处理输入文件，并将其存储在您的本地云存储存储桶中。批处理输入文件可以是CSV或JSONL格式。在本教程中，您将使用JSONL格式。在JSONL文件中，为每个视频的每一行创建一个字典条目。该字典包含以下键值对：

- `content`：视频的云存储路径。
- `mimeType`：内容类型。在我们的示例中，是一个`avi`文件。
- `timeSegmentStart`：要进行预测的视频中的开始时间戳。*注意*，时间戳必须指定为字符串，后面跟着s（秒）、m（分）或h（小时）。
- `timeSegmentEnd`：要进行预测的视频中的结束时间戳。

In [ ]:
import json

from google.cloud import storage

test_filename = "test.jsonl"
gcs_input_uri = BUCKET_URI + "/" + test_filename

data_1 = {
    "content": test_item_1,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}
data_2 = {
    "content": test_item_2,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}


bucket = storage.Client(project=PROJECT_ID).bucket(BUCKET_URI[5:])
blob = bucket.blob(blob_name=test_filename)
data = json.dumps(data_1) + "\n" + json.dumps(data_2) + "\n"
blob.upload_from_string(data)
print(gcs_input_uri)

检查输入内容
检查 `test.jsonl` 的内容。

In [ ]:
! gsutil cat $gcs_input_uri

### 发送批量预测请求

现在您的Vertex AI模型资源已经训练完毕，您可以通过调用`batch_predict()`方法来进行批量预测，参数如下：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `sync`：如果设置为True，调用会阻塞并等待异步批处理作业完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="hmdb",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，等待批量作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以在批量预测作业完成之前阻塞。

In [ ]:
batch_predict_job.wait()

获取预测结果

接下来，从完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的 Cloud 存储输出存储桶中。
调用方法 `iter_outputs()` 来获取生成的结果文件列表。
每个文件以 JSON 格式包含一个或多个预测请求：

- `content`: 预测请求。
- `prediction`: 预测响应。
 - `ids`: 每个预测请求的内部分配的唯一标识符。
 - `displayNames`: 每个类标签的类名。
 - `confidences`: 每个类标签的预测置信度，介于 0 和 1 之间。
 - `timeSegmentStart`: 视频中视频序列开始的时间偏移。
 - `timeSegmentEnd`: 视频中视频序列结束的时间偏移。

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}".replace(
        BUCKET_URI + "/", ""
    )
    data = bucket.get_blob(gfile_name).download_as_string()
    data = json.loads(data)
    print(data)

清理

要清理此项目中使用的所有 Google Cloud 资源，您可以[删除用于教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的单个资源：

- 数据集
- 模型
- AutoML 训练作业
- 批处理作业
- Cloud Storage 存储桶

In [ ]:
# If the bucket needs to be deleted too, please set "delete_bucket" to True
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

# Delete the Cloud storage bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI